In [4]:
import json

with open('politicianNodes.json', 'r') as f:
    data = json.load(f)

for node in data['nodes']:
    node['id'] = node.pop('key')
    #del node['label']
    node.update(node.pop('attributes'))

with open('politicianNodes_modified.json', 'w') as f:
    json.dump(data, f)

In [5]:
import json

with open('politicianNodes_modified.json', 'r') as f:
    data = json.load(f)

for node in data['nodes']:
    node['fx'] = node.pop('x')
    node['fy'] = node.pop('y')

    node['num_followers'] = node.pop('0')
    node['num_following'] = node.pop('1')
    node['name'] = node.pop('2')
    node['screen_name'] = node.pop('3')
    node['party'] = node.pop('4')

    del node['label']


with open('politicianNodes_modified_final.json', 'w') as f:
    json.dump(data, f)